# Проект по распознованию токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

### Имортируем все необходимые инструмены

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import swifter
import spacy
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm
tqdm.pandas()
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer
)
from nltk.stem import WordNetLemmatizer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
nltk.download('stopwords')
from nltk.corpus import (
    stopwords,
    wordnet
)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\piros\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузим данные

In [2]:
try:
    data = pd.read_csv("/datasets/toxic_comments.csv", index_col=0)
except:
    data = pd.read_csv('toxic_comments.csv', index_col=0)

### Откроем первые 5 строк каждой таблицы

In [3]:
data.sample(5)

,text,toxic
125262,"""\nSince QG pointed out the cigalike is hard t...",0
75151,Let me make it perfectly clear... \n\n...that ...,1
48686,"""::::::::That can't be right, because WP:WORDI...",0
72664,"Or, if you need a link to confirm this: []",0
134707,"Tony, I thought WP:CIV was cancelled on Monday...",0


### Посмотрим информацию о данных

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


### Посмотрим баланс у целевого признака

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Наблюдаем дисбаланс**

### Приведём текст к нижнему регистру

In [6]:
data['text'] = data['text'].str.lower()

### Лемматизируем текст в колонке с признаком

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def clear_text(text):
    lists = re.sub(r'[^a-zA-Z ]', ' ', text.lower())
    words = lists.split()
    words = " ".join(words) 
    return words

In [9]:
def lemma_clear(text): 
      
    lemm = nlp(text) 
    lemm = " ".join([token.lemma_ for token in lemm])  
    return " ".join(lemm.split())

In [10]:
data['lemm_text'] = data['text'].apply(clear_text)

In [11]:
data['lemm_text'] = data['lemm_text'].apply(lemma_clear)

In [12]:
data.head()

,text,toxic,lemm_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour I m seem...
2,"hey man, i'm really not trying to edit war. it...",0,hey man I m really not try to edit war it s ju...
3,"""\nmore\ni can't make any real suggestions on ...",0,more I can t make any real suggestion on impro...
4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...


### Подготовим данные к обучению моделей

In [13]:
target = data['toxic']
features = data['lemm_text']

In [14]:
features_train, features_test, target_train, test_target = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

features_valid, features_test, target_valid, test_target = train_test_split(
    features_test, test_target, test_size=0.50, random_state=12345, stratify=test_target)

features_train.shape, features_valid.shape, features_test.shape

((95575,), (31858,), (31859,))

### Уберём в признаках стоп-слова

In [15]:
stopwords = stopwords.words('english')

In [16]:
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))

In [17]:
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)
tf_idf_valid = count_tf_idf.transform(features_valid)

## Обучение

### Модель логической регрессии

In [18]:
scoring = 'f1'
CV = 4

pipeline_logr = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression(random_state=12345))
])

params = {
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': [0.01, 0.1, 1, 10, 100]
}

grid = GridSearchCV(pipeline_logr, cv=CV, n_jobs=-1, param_grid=params ,scoring=scoring)
grid.fit(features_train, target_train)
grid.score(features_test, test_target)

0.7781793150920667

**Модель логической регрессии показала лучший результат:**

***На тестовой выборке 0.77***

### Модель решающего древа

In [19]:
model_tree = DecisionTreeClassifier(
            random_state=12345,
            max_depth=8,
            min_samples_split=10,
            min_samples_leaf=10,
            class_weight='balanced'
            )
model_tree.fit(tf_idf_train, target_train)
predictions = model_tree.predict(tf_idf_test)
f1_score(test_target, predictions)

0.5254382214621633

**К сожалению решающее древо показало плохой результат на тестовой выборке 0.52**

### Модель CatBoostClassifier

In [20]:
best_result = 0
best_depth = 0
best_iterations = 0
for depth in range(2, 10, 2):
    for iterations_i in range(100, 200, 100):
        model_cat = CatBoostClassifier(
                                      random_state=12345,
                                      task_type='CPU',
                                      iterations=iterations_i,
                                      depth = depth,
                                      od_type='Iter'
                                      )
        model_cat.fit(tf_idf_train, target_train)
        predictions = model_cat.predict(tf_idf_test)
        f1 = f1_score(test_target, predictions)
        if best_result < f1:
            best_result = f1
            best_depth = depth
            best_iterations = iterations_i
print(best_result)
print(best_depth)
print(best_iterations)

Learning rate set to 0.5
0:	learn: 0.3561758	total: 246ms	remaining: 24.4s
1:	learn: 0.2811307	total: 333ms	remaining: 16.3s
2:	learn: 0.2583851	total: 422ms	remaining: 13.6s
3:	learn: 0.2467951	total: 512ms	remaining: 12.3s
4:	learn: 0.2388936	total: 601ms	remaining: 11.4s
5:	learn: 0.2331570	total: 688ms	remaining: 10.8s
6:	learn: 0.2290555	total: 778ms	remaining: 10.3s
7:	learn: 0.2258873	total: 868ms	remaining: 9.98s
8:	learn: 0.2198779	total: 953ms	remaining: 9.63s
9:	learn: 0.2165926	total: 1.04s	remaining: 9.41s
10:	learn: 0.2133789	total: 1.13s	remaining: 9.17s
11:	learn: 0.2104942	total: 1.22s	remaining: 8.96s
12:	learn: 0.2077081	total: 1.31s	remaining: 8.76s
13:	learn: 0.2043322	total: 1.4s	remaining: 8.58s
14:	learn: 0.2025165	total: 1.49s	remaining: 8.42s
15:	learn: 0.2005247	total: 1.57s	remaining: 8.26s
16:	learn: 0.1986823	total: 1.66s	remaining: 8.1s
17:	learn: 0.1953375	total: 1.75s	remaining: 7.99s
18:	learn: 0.1938129	total: 1.84s	remaining: 7.86s
19:	learn: 0.19198

In [21]:
model = CatBoostClassifier(
    random_state=12345,
    task_type='CPU',
    iterations=100,
    depth = 8,
    od_type='Iter'                                   
)
predictions = model_cat.predict(tf_idf_valid)
f1_score(target_valid, predictions)


0.7621908127208481

**CatBoostRegressor с показателями:**

***На тестовой выборке F1 = 0.74***

***На валидационной выборке F1 = 0.76***

## Выводы

**Были проведенные исследования данных и были выполненны следующие действия:**

**Данные были загруженны**

**Так же данные были лемматизированны**

**Данные были подготовленны к обучению моделий, а так же из них были удалены стоп-слова**

**Были обучены три модели и они показали следующий результат:**

***Модель логической регрессии - F1 = 0.7791***

***Модель решающего древа - F1 = 0.52***

***Модель CatBoostClassifier:***

***На тестовой выборке F1 = 0.74***

***На валидационной выборке F1 = 0.76***

**Модель логической регрессии показала себя лучшей среди тестируемых c F1 = 0.77**